In [2]:
from datetime import timedelta
from bs4 import BeautifulSoup
import requests
import pandas as pd
import wikipedia
from textblob import TextBlob

wikipedia.set_rate_limiting(True, min_wait=timedelta(milliseconds=100))

Wikipedia documentation: https://wikipedia.readthedocs.io/en/latest/quickstart.html

TextBlobo documentation: https://textblob.readthedocs.io/en/dev/

The function receives the url an get the information and transform it in a dataframe with name link of the article as collumns

The first lines are to transform the information in two different classes in the page in a unique list
The last lines are to separate the information (name and link of article) in two lists, make a dictionary and transform in a pandas dataframe

In [3]:
def page_scrapper(url):
    """Receive the wikipedia project url and returns a dataframe with name, link, summary, photo link and
    tags from the summary's text."""  
    response = requests.get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    """The table has two classes, so the list need to be sum."""
    name_containers_even = html_soup.find_all('tr', class_ = 'list-even')
    name_containers_odd = html_soup.find_all('tr', class_ = 'list-odd')
    name_containers = name_containers_even + name_containers_odd
    
    """Grab name and link from the page's table."""
    name_list = []
    link_list = []
    summary_list=[]
    tags_list=[]

    for item in name_containers:
        name = item.a.text
        link = item.a['href']
        
        try:
            page = wikipedia.WikipediaPage(name)
        except wikipedia.exceptions.PageError:
            continue
            
        name_list.append(name)
        link_list.append(link)
        summary_list.append(page.summary)
            
        """Grab the tags of the wikipedia's article from the summary."""
        tags = TextBlob(page.summary).tags
        tags = [row for row in tags if row[1] in ['NNP', 'NN']]
        tags_list.append(tags)

    return name_list, link_list, summary_list, tags_list

In [4]:
project_list = ['WikiProject_Women', 'Women%27s_History', 'Women_writers', 'Women_scientists', 'Women_artists', 'Women%27s_sport']

In [5]:
for project_name in project_list:

    """Include the information about the project and page that will be used."""
    offset_number = 1
    limit_number = 1000

    """Create the url that will be used in the function page_scrapper."""
    while True:
        base = 'https://tools.wmflabs.org/enwp10/cgi-bin/list2.fcgi?run=yes'
        project = f'&projecta={project_name}'
        limit = f'&limit={limit_number}'
        offset = f'&offset={offset_number}'

        url = base + project + limit + offset
        print(url)
        
        #name_list, link_list, summary_list, tags_list = page_scrapper(url)

        offset_number += limit_number
        if len(name_list) == 0:
            break

https://tools.wmflabs.org/enwp10/cgi-bin/list2.fcgi?run=yes&projecta=WikiProject_Women&limit=10&offset=1


NameError: name 'name_list' is not defined